<a href="https://colab.research.google.com/github/npd2013/Mypublicmodels/blob/main/7_x_random_summary_generation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install needed packages 
#!pip install  gensim  #has word2vec model built, using english 

!pip install rouge
!pip install indic-nlp-library


In [ ]:
from google.colab import files #upload the files from local machine on the colab environment 
uploaded = files.upload()


In [ ]:
!cp "/content/drive/My Drive/Colab Notebooks/guj_final_preporcess-v6.py" "guj_final_preporcess.py" #The preprocessing modules designed
!cp "/content/drive/My Drive/Colab Notebooks/Evaluation.py" "Evaluation.py"   # the evaluation module designed


In [ ]:
import io
import csv
from Evaluation import *
import random

import pandas as pd 
from guj_final_preporcess import * #to get the required preporcessing modules -from self made pipeline


In [ ]:
#take all the needed inputs from user 
basename=input('Enter the basic name of file(without txt),other names wud be autogenerated  ')
filename=basename+'.txt'
gold_filename=basename+'-gold.txt'

summary_filename=basename+'-randomsummary.txt' # example file1-gold-system-summary.txt
analysefile_name=basename+'-analysis-randomsummary.txt'
rougescorefile="Rougeresults--randomsummarygeneration .csv"


Enter the basic name of file(without txt),other names wud be autogenerated  story50


In [ ]:
#read the input original file . 

guj_clean_html(filename,filename)# text cleaning remove the html tags and overwrite the original file 
guj_clean_html(gold_filename,gold_filename)# clean the html tags from gold file 


guj_clean_extra(filename,filename)#remove the extra symbols like multiple punctuations and overwrite original file
guj_clean_extra(gold_filename,gold_filename)

#Display the given data in input files 
with io.open(filename,'r',encoding='utf-8') as f: 
		  lines=f.read()#get all the lines from file
analyzedfile_obj=open(analysefile_name,mode='w',encoding='UTF-8')
analyzedfile_obj.write("\n\n The original file is :------\n")
analyzedfile_obj.write(lines)

with io.open(gold_filename,'r',encoding='utf-8') as f: 
		  lines=f.read()#get all the lines from file

analyzedfile_obj.write("\n\n The gold summary provided by expert  is -->\n")    
analyzedfile_obj.write(lines)


###################################################################
########################  Step-1 ##################################
#####################  cleaning phase  ########################

#doing cleaning of file 
#remove newlines and other unnecessary data and clean it for feature extraction 
#to replace the data as per our format, replacing abbreviations dot with '*'
normfile="temp1.txt"
norm_gold_file="temp2.txt"


guj_clean_text(filename,normfile) 

guj_clean_text(gold_filename,norm_gold_file)

#Now display the stats and make corpora after cleaning file and removing new lines . 
normcorpora,totalnormsentences=guj_corpus_generate(normfile)  
goldcorpora,totalnormgoldsentences=guj_corpus_generate(norm_gold_file) 



In [ ]:
 rougescorefilehandle=open(rougescorefile,mode='a',encoding="utf-8") #to store only the rouge scores for later analysis 
 resultvar = csv.writer(rougescorefilehandle, lineterminator='\n')		#to store new row for each f  


################# ######Step4#############################
################ Find top sentences  #################

#rank and sort the sentences  based on individual sentences score 

 percent=(totalnormgoldsentences/totalnormsentences)*100 #ideal ratio as per gold text given 
 n=totalnormgoldsentences#extract sentences as per total sentences in gold summary 

 resultlist=[]#to store the rouge score data

 resultlist.append(filename+",") #mention the source file name for record
 
 
  ##########################################################
################# ######Step6#############################
################ Generate summary  #################
##########################################################
 summary_random_numbers=[] #store the random sentence numbers to be put in summary 

 summary_random_numbers=random.sample(range(1, totalnormsentences), n) #get n random numbers to map to sentence numbers  
 print(summary_random_numbers)
 linenumber=0

 with io.open(summary_filename,'w',encoding='utf-8') as w: 
  for sent in normcorpora:
   
   if linenumber  in summary_random_numbers:
     w.write(sent)
     w.write("\n")
   linenumber+=1 #increment line number when referening next line from corpora
     

 analyzedfile_obj.write("\n\n ***** The summary geneated has the following random numbered sentences ---->>\n {0}".format(summary_random_numbers))
 
  
norm_summary_filename="t.txt"
    
################ Evaluate summary  #################
 guj_remove_newlines(norm_gold_file,norm_gold_file)
 guj_remove_newlines(summary_filename,norm_summary_filename)
 rougescores=evalate_files(norm_gold_file,norm_summary_filename)#get the evaluation of generated summary after post processing based on rouge score
 for key in rougescores.keys():
     resultlist.append("%s,%s"%(key,rougescores[key]))
 resultvar.writerow(resultlist)   
 analyzedfile_obj.write("\n **********************Rouge scores based analysis for {0} % extraction is as follows*********************** ".format(percent))
 analyzedfile_obj.write("\n\n The rouge scores on summary file are \n {0}".format(rougescores,2)) #store the results 
 basicfile="original.txt"
  #guj_clean_text(filename,basicfile) # Get the original file in cleaned form for computing compression ratio. 
 guj_remove_newlines(filename,basicfile) 
  
 sent_compression,word_compression,totaloriginalsentences,totaloriginalwords,totalsumsentences,totalsumwords=compression_statistics(summary_filename,basicfile) #to store the compression ratio analysis
 analyzedfile_obj.write("\n **********************Compression Analysis at {0} % extraction is as follows(based on original file) *********************** ".format(percent))
 analyzedfile_obj.write("\n Total sentences in original text were {0} ".format(totaloriginalsentences)) 
 analyzedfile_obj.write("\n Total sentences in summary text are {0} ".format(totalsumsentences))
 analyzedfile_obj.write("\n Sentences compression ratio is {0} \n".format(sent_compression,2))
 analyzedfile_obj.write(" \n\n Total words in original text were {0} ".format(totaloriginalwords))
 analyzedfile_obj.write(" \n total words in summary text are {0} ".format(totalsumwords))
 analyzedfile_obj.write("\n Words compression ratio is {0} ".format(word_compression,2)) 



[24, 45, 33, 16, 20, 6, 42, 71, 23, 51, 75, 81, 11, 39, 76, 32, 41, 25, 4, 63, 72, 77, 60, 80, 9, 66, 61, 12, 34, 56, 52, 30, 57, 58, 19, 35, 44, 13, 10, 47, 3, 79, 67, 31, 21, 54, 59, 65, 8, 28, 46, 14, 15, 18, 22, 68, 36, 17, 26, 5, 40, 48, 2, 73, 37, 38, 82, 74]
{'rouge-1': {'f': 0.8707753429133176, 'p': 0.8599476439790575, 'r': 0.8818791946308725}, 'rouge-2': {'f': 0.8400796234015911, 'p': 0.8296199213630406, 'r': 0.8508064516129032}, 'rouge-l': {'f': 0.8855721343034876, 'p': 0.8864541832669323, 'r': 0.8846918489065606}}


48

In [ ]:
analyzedfile_obj.close()

rougescorefilehandle.close()

In [ ]:
## code to download all data at end of execution .Uncomment it after all experiments done. 

from google.colab import files

!zip -r /content/txtfiles.zip *.txt
files.download("txtfiles.zip")
!zip -r /content/csvfiles.zip *.csv
files.download("csvfiles.zip")
!zip -r /content/codesfiles.zip *.py
files.download("codesfiles.zip")



  adding: original.txt (deflated 76%)
  adding: story10-analysis-randomsummary.txt (deflated 80%)
  adding: story10-gold.txt (deflated 71%)
  adding: story10-randomsummary.txt (deflated 72%)
  adding: story10-topic.txt (deflated 17%)
  adding: story10.txt (deflated 72%)
  adding: story11-analysis-randomsummary.txt (deflated 82%)
  adding: story11-gold.txt (deflated 75%)
  adding: story11-randomsummary.txt (deflated 75%)
  adding: story11-topic.txt (deflated 2%)
  adding: story11.txt (deflated 76%)
  adding: story12-analysis-randomsummary.txt (deflated 83%)
  adding: story12-gold.txt (deflated 78%)
  adding: story12-randomsummary.txt (deflated 76%)
  adding: story12-topic.txt (deflated 6%)
  adding: story12.txt (deflated 78%)
  adding: story13-analysis-randomsummary.txt (deflated 83%)
  adding: story13-gold.txt (deflated 77%)
  adding: story13-randomsummary.txt (deflated 76%)
  adding: story13-topic.txt (deflated 6%)
  adding: story13.txt (deflated 77%)
  adding: story14-analysis-random

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  adding: Rougeresults--randomsummarygeneration .csv (deflated 68%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  adding: Evaluation.py (deflated 64%)
  adding: guj_final_preporcess.py (deflated 75%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>